* Data from https://opendata.dwd.de/climate_environment/CDC/event_catalogues/germany/precipitation/CatRaRE_v2023.01/data/CatRaRE_2001_2022_W3_Eta_v2023_01.gdb.zip

In [2]:
using GeoDataFrames,GeoJSON,ArchGDAL

In [3]:
layer=ArchGDAL.getlayer(ArchGDAL.read("CatRaRE_2001_2022_W3_Eta_v2023_01.gdb"),1)

Layer: CatRaRE_2001_2022_W3_Eta_EventZones_v2023_01
  Geometry 0 (Shape): [wkbMultiPolygon], MULTIPOLYGON (((-161...), ...
     Field 0 (CatVersion): [OFTString], W3_Eta_2023.01, W3_Eta_2023.01, ...
     Field 1 (KEY_FIELD): [OFTString], W3_Eta_1, W3_Eta_2, W3_Eta_3, ...
     Field 2 (Event_ID): [OFTInteger], 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
     Field 3 (Start_Time): [OFTReal], 2.0010102035e11, 2.0010104035e11, ...
     Field 4 (End_Time): [OFTReal], 2.0010102075e11, 2.0010106035e11, ...
...
 Number of Fields: 69

In [4]:
using DataFrames

In [5]:
df=DataFrame(layer);

In [6]:
using GLMakie,GeoMakie

In [7]:
proj_string="+proj=stere +lat_0=90 +lat_ts=90 +lon_0=10 +k=0.93301270189 +x_0=0 +y_0=0 +a=6370040 +b=6370040  +no_defs"

"+proj=stere +lat_0=90 +lat_ts=90 +lon_0=10 +k=0.93301270189 +x_0=0 +y_0=0 +a=6370040 +b=6370040  +no_defs"

In [8]:
latlon_string="+proj=longlat +datum=WGS84 +no_defs +type=crs"

"+proj=longlat +datum=WGS84 +no_defs +type=crs"

In [9]:
using Proj

In [10]:
using GeoFormatTypes

In [11]:
ArchGDAL.createcoordtrans(ArchGDAL.importPROJ4(proj_string), ArchGDAL.importPROJ4(latlon_string)) do transform
    for x in eachrow(df)
    ArchGDAL.transform!(x.Shape, transform)
    end
end

In [12]:
boundary=GeoDataFrames.read("../Kreisgrenzen_2021_mit_Einwohnerzahl.geojson");

In [13]:
#dadi=boundary[(boundary.GEN.=="Darmstadt").|(boundary.GEN.=="Darmstadt-Dieburg"),:]
dadi=boundary[(boundary.GEN.=="Darmstadt"),:]

Row,geometry,OBJECTID,OBJID,BEGINN,AGS,GEN,BEZ,IBZ,BEM,SN_L,SN_R,SN_K,SN_V1,SN_V2,SN_G,FK_S3,NUTS,WSK,EWZ,KFL,SHAPE_Length,SHAPE_Area
,IGeometr…,Int32,String,DateTime,String,String,String,Int32,String,String,String,String,String,String,String,String,String,DateTime,Int32,Float64,Float64,Float64
1,Geometry: wkbMultiPolygon,117,DEBKGVG200000CAD,2022-01-05T00:00:00,06411,Darmstadt,Kreisfreie Stadt,40,--,06,4,11,00,00,000,R,DE711,1977-01-01T00:00:00,159631,122.07,0.722813,0.0153565


In [14]:
intersects=Vector{ArchGDAL.IGeometry{ArchGDAL.wkbMultiPolygon}}()
colors=[]
for x in dadi.geometry

    for y in eachrow(df)
        if ArchGDAL.intersects(x,y.Shape)
            intersects=vcat(intersects,y.Shape)
            push!(colors,y.RRmax)
        end
    end
end

In [21]:
fig=Figure()
q=GridLayout(fig[1,1])
Label(q[1,1],text="Where should you live in Darmstadt \nif you don't want to be caught in heavy rainfall?",fontsize=30)
Label(q[2,1],text="Based on the events from CatRaRe 2001-2022",fontsize=20)
ax=GeoAxis(fig[2,1],lonlims=(8.2,9.1),latlims=(49.5,50.2),dest=latlon_string)
p=poly!(ax,GeoMakie.geo2basic(intersects),color=colors,colormap=cgrad(:bamako,alpha=0.5,rev=true),strokecolor=:gray,strokewidth=1)
poly!(ax,GeoMakie.to_multipoly.(GeoMakie.geo2basic.(dadi.geometry)),color=:transparent,strokecolor=:black,strokewidth=3)
#poly!(ax,GeoMakie.to_multipoly.(GeoMakie.geo2basic.(boundary.geometry)),color=:transparent,strokecolor=:black,strokewidth=0.1)
hidespines!(ax)
hidedecorations!(ax, grid=true,ticklabels=false)
Colorbar(fig[2,2],p,vertical=true,label="Maximum rain in eventzone in mm")
arrows!(ax,[8.55],[50.1],[1],[-1.5],lengthscale=0.12)
text!(Point2f(8.55,50.1),text="Here :D",fontsize=20)
Label(fig[3,1],text="Data: © Deutscher Wetterdienst\nGeodata: © GeoBasis-DE / BKG, Statistisches Bundesamt (Destatis) (2022)",justification=:left)
Makie.resize_to_layout!(fig)
fig


In [22]:
save("catrare_darmstadt.png",fig,px_per_unit=2)